# Vision Transformer (ViT)
번 실습에서는 Vision Transformer (ViT) 아키텍처를 직접 구현해보겠습니다. 

ViT는 원래 자연어 처리에서 주로 사용되던 Transformer 아키텍처를 이미지에 적용하여 이미지 분류 문제에서 뛰어난 성능을 발휘하도록 한 모델입니다. 

실습에서는 ViT를 구성하는 핵심 모듈들을 단계별로 구현해봄으로써, ViT 아키텍처의 구조와 원리를 이해하게 됩니다. 각 모듈이 어떤 역할을 하고, 최종적으로 어떻게 조합되어 이미지 분류를 수행하는지 알아보도록 합시다.

이번 실습은 다음과 같은 주요 모듈들을 순차적으로 구현하는 것으로 이루어져 있습니다.
- MultiHeadSelfAttention : 이미지의 패치(patch)들 간의 상호작용을 학습하며, 각 패치의 정보가 다른 패치들의 정보와 자연스럽게 통합(integrate) 됩니다. 이를 통해 각 패치는 이미지 전체 맥락을 반영한 더욱 풍부하고 정교한 표현(representation)을 학습할 수 있습니다.
- FeedForwardNetwork : Self-Attention에서 나온 정보를 비선형 변환하여 더욱 복잡한 표현을 학습하게 합니다.
- TransformerEncoder : Self-Attention과 FeedForwardNetwork를 결합하여 하나의 Transformer 인코더 블록을 구성합니다. 여러 인코더 블록을 쌓아 ViT의 깊이를 조절할 수 있습니다.
- ImagePatchifier : 이미지를 여러 작은 패치(patch)로 분할하여 Transformer의 입력으로 사용할 수 있는 형태로 변환합니다. 
- PatchEmbedding : 분할된 패치를 벡터로 변환하여 Transformer가 처리할 수 있는 임베딩(embedding) 형태로 만듭니다. ViT에서는 각 패치가 단어처럼 취급되어 입력됩니다.
- ViT : 위의 모듈들을 종합하여 최종 Vision Transformer 모델을 구현합니다. 완성된 모델은 입력된 이미지 패치들 간의 관계를 학습하고, 최종적으로 이미지를 분류하는 결과를 제공합니다.

Original paper : An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale ([link](https://arxiv.org/pdf/2010.11929))

In [ ]:
import numpy as np

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models
import torchvision.transforms.v2 as transforms

import wandb
from tqdm import tqdm
import matplotlib.pyplot as plt

from training_utilities import train_loop, evaluation_loop, save_checkpoint, load_checkpoint, load_cifar10_dataloaders

## Multi-Head Self-Attention

Transformer의 핵심 모듈로, 이미지의 패치(patch)들 간의 상호작용을 학습하며, 각 패치의 정보가 다른 패치들의 정보와 자연스럽게 통합(integrate) 됩니다. 이를 통해 각 패치는 이미지 전체 맥락을 반영한 더욱 풍부하고 정교한 표현(representation)을 학습할 수 있습니다.

### 구성요소

- Linear Projections: 입력 임베딩(embedding)을 queries, keys, values로 변환하는 학습가능한 선형 레이어
- Attention Mechanism: queries와 keys를 이용하여 Attention score를 계산하고, 이를 values값에 적용합니다. 이 과정에서 패치 간 중요한 정보가 통합됩니다.
- Output Projection: 모든 attention head로부터 나온 출력값을 합쳐 하나의 최종 결과를 생성하는 선형 레이어입니다.

### 계산과정

Attention Score는 아래의 수식을 통해 계산됩니다:
$$ Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$$
여기서 $d_k$는 queries, keys, values의 차원의 크기 (hidden dimension size)입니다 

실제 계산에서는 $d_{model}$크기의 hidden vector에 한번의 attention을 적용하는 것 보다 (single head) 서로 다른 query, key, value 선형변환을 $h$번 적용하는것이 더 성능이 뛰어납니다. (Multi head attention이라고 불림, $h$는 head의 수(number of heads))

$$ MultiHead(Q, K, V) = Concat(head_1, ..., head_h)W^O$$
where $ head_i = Attention(QW_i^Q,KW_i^K, VW_i^V)$
 - $d_{model} = h \times d_k$
 - $W_i^Q,W_i^K, W_i^V$ are linear projection matrices of shape $(d_{model}, d_k)$
 - $W^O$ are linear projection matrix of shape $(hd_k, d_{model}) = (d_{model}, d_{model})$

### <mark>실습 </mark> 아래 구현 과정에 따라 `MultiHeadSelfAttention`을 완성하세요.

1. Queries, Keys, Values 계산 : 입력 텐서 `x`에 linear layer를 적용하여 queries `Q`, keys `K`, values `V` 들을 계산합니다.

2. multiple heads로 쪼개기: `Q`, `K`, `V`를 변환하여 `(batch_size, num_heads, seq_length, head_dim)`의 shape을 가지도록 변환합니다.
   - [torch.Tensor.view](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html), [torch.reshape](https://pytorch.org/docs/stable/generated/torch.reshape.html): 텐서의 shape 변경
   - [torch.permute](https://pytorch.org/docs/stable/generated/torch.permute.html): 텐서의 차원 순서를 재배열

3. Scaled Dot-Product Attention 계산:
   - $Q$와 $K^T$의 dot product를 계산하여 attention score를 계산한다.
     - [torch.transpose](https://pytorch.org/docs/stable/generated/torch.transpose.html)
     - [torch.matmul](https://pytorch.org/docs/stable/generated/torch.matmul.html)
   - attention scores를 `self.head_dim`의 제곱근으로 나누어 스케일링해주어 안정적인 학습을 돕습니다.
   - `torch.softmax` 함수를 이용하여 attention weight를 계산합니다.
   - attention weight에 dropout layer를 적용한다.

4. Attention Output:
   - attention weights와 values `V`를 곱하여 각 head에 대한 attention 출력값을 계산합니다.
   - 모든 head의 attention출력값을 concat합니다 (`permute`와 `view`함수 이용).
   - concat된 output에 $W^O$에 대응하는 선형 변환(`self.fc`)을 수행합니다.


In [ ]:
class MultiHeadSelfAttention(nn.Module):
    """
    Multi-Head Self Attention Module.
    """
    def __init__(self, hidden_dim, num_head, dropout_prob = 0.1):
        super().__init__()
        self.hidden_dim = hidden_dim            # d_model, total hidden dimension
        self.num_head = num_head                # h
        self.head_dim = hidden_dim // num_head  # d_k
        assert self.head_dim * num_head == hidden_dim, "hidden_dim must be divisible by num_head."

        self.query_projection = nn.Linear(hidden_dim, hidden_dim)
        self.key_projection = nn.Linear(hidden_dim, hidden_dim)
        self.value_projection = nn.Linear(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        batch_size, seq_length, hidden_dim = x.size() 

        # (batch_size, seq_len, num_head, head_dim)
        Q = self.query_projection(x).view(batch_size, seq_length, self.num_head, self.head_dim)
        K = self.key_projection(x).view(batch_size, seq_length, self.num_head, self.head_dim)
        V = self.value_projection(x).view(batch_size, seq_length, self.num_head, self.head_dim)

        # (batch_size, num_head, seq_len, head_dim)
        Q = Q.permute(0, 2, 1, 3)
        K = K.permute(0, 2, 1, 3)
        V = V.permute(0, 2, 1, 3)

        ##### YOUR CODE START #####

        ##### YOUR CODE END #####

        return out

In [ ]:
hidden_dim = 64       # d_model
num_head = 8          # Number of heads
seq_length = 10       # Sequence length
batch_size = 32       # Batch size

attention_module = MultiHeadSelfAttention(hidden_dim=hidden_dim, num_head=num_head)
x = torch.randn(batch_size, seq_length, hidden_dim)
output = attention_module(x)
print("Output shape:", output.shape)

assert output.shape == (batch_size, seq_length, hidden_dim), "Output shape is incorrect!"

## Position-wise Feed-Forward Networks
`MultiHeadSelfAttention` 모듈의 출력값에 fully connected feed-forward network(FFN)를 적용합니다. 이 네트워크는 각 위치(position)별로 독립적으로 작동하며, 동일한 변환을 각 위치에 개별적으로 수행합니다. FFN은 모델이 더 복잡한 패턴을 학습할 수 있도록 돕습니다.

$$ FFN(x) = GELU(xW_1+b_1)W_2+b_2$$

구성요소:
 - 두개의 Linear Layers: 임베딩 차원(embedding diemmension)을 확장하였다가 다시 원래 차원으로 축소해 줍니다.
 - Activation Function: GELU non-linear activation 함수를 적용하여 모델이 복잡한 패턴을 학습할 수 있도록 합니다.
 - Dropout Layer for regularization

### <mark>실습 </mark> 아래 구현 과정에 따라 `FeedForwardNetwork`을 구현하세요.

- linear layer that maps from `hidden_dim` to `feedforward_dim`.
- GELU activation ([troch.nn.GELU](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html)).
- dropout layer
- A linear layer that maps back from `feedforward_dim` to `hidden_dim`.


In [ ]:
class FeedForwardNetwork(nn.Module):
    """
    Position-wise Feed-Forward Networks
    """
    def __init__(self, hidden_dim, feedforward_dim, dropout_prob = 0.1):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(hidden_dim, feedforward_dim),
            ##### YOUR CODE START #####

            ##### YOUR CODE END #####
        )

    def forward(self, x):
        out = self.net(x)
        
        return out

In [ ]:
hidden_dim = 64
feedforward_dim = 1024
seq_length = 10       # Sequence length
batch_size = 32       # Batch size

ffn = FeedForwardNetwork(hidden_dim=hidden_dim, feedforward_dim=feedforward_dim, dropout_prob = 0.2)
x = torch.randn(batch_size, seq_length, hidden_dim)
output = ffn(x)
print("Output shape:", output.shape)

assert output.shape == (batch_size, seq_length, hidden_dim), "Output shape is incorrect!"

## TransformerEncoder

<img src="resources/vit_encoder.png" style="width:200px;">

`MultiHeadSelfAttention`, `FeedForwardNetwork`, layer normalization을 모두 조합하여 Transformer encoder를 구현합니다.

구성 요소:
- Layer Normalization (LN): attention과 feed-forward 레이어 <u>전에</u> 적용해줍니다. 이를 Pre-LN방식이라 부르며, 특히 깊은 모델에서 학습 안정성을 높이는 데 효과적입니다.
- Dropout Layers for regularization : 각 레이어 출력값에 dropout을 적용하여 과적합을 방지합니다.
- Residual Connections: 각 레이어의 출력값에 입력값을 더해준다. 이를 통해 학습 안정성과 정보 흐름을 개선할 수 있습니다.

### <mark>실습 </mark> 아래 구현 과정에 따라 `TransformerEncoder`을 완성하세요.

1. First Sub-layer (Attention):
    - 입력값에 layer normalization을 적용한다 ([nn.LayerNorm](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html) 이용).
    - `MultiHeadSelfAttention` 모듈에 통과시킨다.
    - 출력값에 dropout을 적용합니다.
    - dropout이 적용된 출력값에 이 sub-layer의 입력값을 더해줍니다 (residual connection).

2. Second Sub-layer (Feed-Forward Network):

    - 첫번째 sub-layer출력값에 layer normalization을 적용한다.
    - `FeedForwardNetwork` module에 통과시킨다
    - feed-forward 출력값에 dropout을 적용합니다.
    - 출력값에 이 sub-layer의 입력값을 더해준다


In [ ]:
class TransformerEncoder(nn.Module):
    """
    Single Transformer Encoder Layer.
    """
    def __init__(self, hidden_dim, num_head, feedforward_dim, dropout_prob=0.1):
        super().__init__()

        self.norm1 = nn.LayerNorm(hidden_dim)
        self.mha = MultiHeadSelfAttention(hidden_dim, num_head, dropout_prob = dropout_prob)
        self.dropout1 = nn.Dropout(dropout_prob)
        self.norm2 = nn.LayerNorm(hidden_dim)
        self.ffn = FeedForwardNetwork(hidden_dim, feedforward_dim, dropout_prob = dropout_prob)
        self.dropout2 = nn.Dropout(dropout_prob)

    def forward(self, x):
        ##### YOUR CODE START #####

        ##### YOUR CODE END #####

        return out


In [ ]:
hidden_dim = 64
num_head = 8
feedforward_dim = 1024
seq_length = 10       # Sequence length
batch_size = 32       # Batch size

encoder = TransformerEncoder(hidden_dim=hidden_dim, num_head = num_head, feedforward_dim=feedforward_dim, dropout_prob = 0.2)
x = torch.randn(batch_size, seq_length, hidden_dim)
output = encoder(x)
print("Output shape:", output.shape)

assert output.shape == (batch_size, seq_length, hidden_dim), "Output shape is incorrect!"

## ImagePatchifier

입력 이미지를 서로 겹치지 않는 패치(patch)들로 분할하고, 이를 Transformer에서 처리할 수 있는 시퀀스(sequence)형태의 입력을 변환하는 역할을 합니다. 

$(C, H, W)$ 차원의 이미지는 $(N, C \times P^2)$ 형태로 변환됩니다.
- $(H, W)$는 원본 이미지의 해상도(높이, 너비)이다.
- $C$는 채널 수, $(P, P)$는 각 패치의 해상도이다.
- $N = HW/P^2$는 생성되는 패치의 수 입니다.

<img src="resources/vit_model.png" style="width:600px;">

### <mark>실습 </mark> 아래 구현 과정에 따라 `ImagePatchifier`을 완성하세요.
1. Patch Extraction: 이미지를 (patch_size, patch_size) 크기의 패치들로 나눕니다.
    - [torch.nn.Unfold](https://pytorch.org/docs/stable/generated/torch.nn.Unfold.html)함수 이용
    - 입력 shape : (batch_size, num_channels, height, width)
    - 출력 shape : (batch_size, num_channels, num_patches_y, num_patches_x, patch_size, patch_size)
2. Flattening: 각 patch들을 Flatten하여 시퀀스 형태로 변환합니다.
    - `view`, `reshape`, `permute`함수를 이용하여 텐서의 형태를 변경합니다.
    - 입력 shape : (batch_size, num_channels, num_patches_y, num_patches_x, patch_size, patch_size)
    - 출력 shape : (batch_size, num_patches, num_channels * patch_size * patch_size)
    - 주의: heigth-to-width 순서를 유지하세요.

In [ ]:
class ImagePatchifier(nn.Module):
    """
    Divides an image into patches and flatten
    """
    def __init__(self, image_size, patch_size):
        super().__init__()

        assert image_size % patch_size == 0, "Image dimensions must be divisible by the patch size."
        self.patch_size = patch_size
        self.num_patches = (image_size // patch_size) ** 2

    def forward(self, x):
        """
        Args:
            Tensor of shape (batch_size, num_channels, height, width)
        Returns:
            Tensor of shape (batch_size, num_patches, num_channels * patch_size * patch_size)
        """
        B, C, H, W = x.shape
        x = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size) #(batch_size, num_channels, num_patches_y, num_patches_x, patch_size, patch_size)
        
        ##### YOUR CODE START #####

        ##### YOUR CODE END #####
        return out

In [ ]:
image_size = 224
patch_size = 16
batch_size = 64

patchfier = ImagePatchifier(image_size, patch_size)
x = torch.randn(batch_size, 3, image_size, image_size)
output = patchfier(x)
print("Output shape:", output.shape)

assert output.shape == (batch_size, 14 * 14, 3 * 16 * 16), "Output shape is incorrect!"

In [ ]:
def visualize_patches(image, patch_size):
    """
    Visualizes the patches of an image.
    Args:
        image: Tensor of shape (C, H, W)
        patch_size: int
    """
    patches = ImagePatchifier(224, patch_size)(image)[0]
    num_patches = patches.size(0)
    grid_size = int(np.sqrt(num_patches))
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(grid_size*2, grid_size*2))
    for idx, patch in enumerate(patches):
        row = idx // grid_size
        col = idx % grid_size
        ax = axes[row, col]
        
        patch = patch.view(-1, patch_size, patch_size).permute(1, 2, 0).numpy()
        ax.imshow(patch)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


from PIL import Image

img = Image.open('resources/n01580077_1031.JPEG')
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
img_tensor = transform(img).unsqueeze(0)  # Shape: (B, C, H, W)

visualize_patches(image = img_tensor, patch_size = 16)

## PatchEmbedding
이미지 패치들을 임베딩 벡터(embedding vector)로 변환한 후, 위치 정보(positional embedding)와 [class] 토큰을 추가합니다.

`ImagePatchifier`에서 시퀀스 형태로 변환된 이미지 패치들을 $[x_p^1; x_p^2; ... ; x_p^N]$이라 하겠습니다. ($N$은 패치 수)

1. Patch embedding

각 이미지 패치에 학습 가능한 선형 변환 행렬 $E$를 곱하여 tranformer에서 사용할 `embedding_dim` $D$으로 변환합니다. 이 변환은 각 패치를 고정된 크기의 벡터로 바꾸어, Transformer가 이미지 패치를 토큰처럼 다룰 수 있도록 합니다.

$$[x_p^1E; x_p^2E; ... ; x_p^NE]$$
where $E$ is embedding projection matrix of shape $(P^2\times C, D)$, $P$는 patch_size, $C$는 채널 수(num_channels)


2. `[class]` token

BERT의 `[class]` 토큰(token)과 유사하게 새로운 학습 가능한 벡터 $x_{class}$를 패치 임베딩의 맨 앞에 추가해줍니다.
$$[x_{class}; x_p^1E; x_p^2E; ... ; x_p^NE]$$

이 `[class]` 토큰은 Transformer encoder를 거치며 이미지 전체의 정보를 요약하는 임베딩으로 학습됩니다. 최종적으로, Transformer Encoder의 출력에서 `[class]` 토큰 해당하는 벡터는 이미지 전체를 대표하는 임베딩 값(image representation)으로 사용되며, 여기에 MLP 헤드를 연결하여 이미지 분류 작업을 수행합니다. 

<img src="resources/vit_model.png" style="width:400px;">

3. Position embeddings

이미지의 위치 정보를 보존하기 위해 학습가능한(learnable) Positional Embedding을 더해줍니다. 이는 모델이 각 패치의 위치를 인식할 수 있도록 하여, 이미지의 공간적 구조를 유지하는 데 도움을 줍니다.
$$[x_{class}; x_p^1E; x_p^2E; ... ; x_p^NE] + E_{pos}$$

여기서 $E_{pos}$는 $(N+1, D)$의 shape을 가집니다.

### <mark>실습 </mark> 아래 구현 과정에 따라 `PatchEmbedding`을 완성하세요.
- Linear Projection: 각 패치에 linear 레이어를 적용하여 `embedding_dim`차원을 가지는 embedding vector를 얻습니다.
- [Class] Token: 이미지 전체를 대표하는 learnable [class] 토큰을 앞선 embedding vector의 앞에 concat합니다.
- Positional Embeddings: 패치들의 위치 정보를 보존하기 위한 Learnable embedding을 더해줍니다.

`nn.Parameter`를 이용해 learnable parameter를 만들어 줍니다.

In [ ]:
class PatchEmbedding(nn.Module):
    """
    Embeds patches, adds classification token and positional embeddings.
    """
    def __init__(self, num_patches, patch_size, embedding_dim, num_channels=3):
        super().__init__()
        self.num_patches = num_patches
        self.embedding_dim = embedding_dim
        self.patch_size = patch_size
        self.num_channels = num_channels

        # Linear projection of flattened features for each patch
        self.linear_projection = nn.Linear(num_channels * patch_size * patch_size, embedding_dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, embedding_dim))
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, embedding_dim))

    def forward(self, patches):
        """
        Args:
            patches: Tensor of shape (batch_size, num_patches, num_channels * patch_size * patch_size)
        Returns:
            embeddings: Tensor of shape (batch_size, num_patches + 1, embedding_dim)
        """
        batch_size = patches.size(0)

        # Project patches to embedding dimension
        patch_embeddings = self.linear_projection(patches)  # (batch_size, num_patches, embedding_dim)

        # Expand class token to batch size
        class_token = self.cls_token.expand(batch_size, -1, -1)  # (batch_size, 1, embedding_dim)

        ##### YOUR CODE START #####
        # Concatenate class token with patch embeddings

        # Add positional embeddings
        
        ##### YOUR CODE END #####

        return embeddings

## ViT (Vision Transformer)
지금까지 구현한 모듈들을 모두 조합하여 ViT 아키텍쳐를 완성합니다.

### <mark>실습 </mark> 아래 과정에 따라 `ViT`를 완성하세요.
1. Patchification: `ImagePatchifier`를 사용해 이미지를 시퀀스 형태의 패치들로 변홥합니다.
2. Embedding: `PatchEmbedding`모듈을 사용하여 patch embedding을 얻습니다,
3. Transformer Encoders: `TransformerEncoder` 레이어를 `num_transformer_layers`만큼 통과시킵니다.
4. Classification Head
    - [class] token에 대응되는 Transformer Encoder의 출력값을 추출하여, 이미지 전체를 대표하는 임베딩으로 사용합니다.
    - 해당 임베딩에 linear layer를 적용하여 이미지 분류를 위한 logit 값을 얻습니다.

In [ ]:
class ViT(nn.Module):
    """
    Vision Transformer (ViT) model.
    """
    def __init__(
        self,
        image_size=224,
        num_channels=3, 
        patch_size=16,
        num_classes=1000,
        hidden_dim=768,
        num_transformer_layers=12,
        num_head=12,
        feedforward_dim=3072,
        dropout_prob=0.1,
    ):
        super().__init__()

        self.patchifier = ImagePatchifier(image_size, patch_size)
        self.patch_embedding = PatchEmbedding(self.patchifier.num_patches, patch_size, hidden_dim, num_channels)

        self.dropout = nn.Dropout(dropout_prob)

        self.transformer_encoder = nn.Sequential(
            *[
                TransformerEncoder(hidden_dim, num_head, feedforward_dim, dropout_prob)
                for _ in range(num_transformer_layers)
            ]
        )

        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, images):
        """
        Args:
            images: Tensor of shape (batch_size, num_channels, image_height, image_width)
        Returns:
            logits: Tensor of shape (batch_size, num_classes)
        """
        patches = self.patchifier(images) # (batch_size, num_patches, num_channels * patch_size * patch_size)
        embeddings = self.patch_embedding(patches)  # (batch_size, 1 + num_patches, hidden_dim)
        embeddings = self.dropout(embeddings)
        
        x = self.transformer_encoder(embeddings) # (batch_size, num_patches + 1, embedding_dim)
        x = self.layer_norm(x)

        ##### YOUR CODE START #####

        ##### YOUR CODE END #####
  
        return logits 

In [ ]:
model = ViT()
assert model(torch.randn(4, 3, 224, 224)).shape == torch.Size((4, 1000)), "output shape does not match"
assert sum(p.numel() for p in model.parameters()) == 86567656, "Number of model parameter does not match"

print("\033[92m All test passed!")

## Training

In [14]:
def get_model(model_name, num_classes, config):
    if model_name == "ViT":
        model = ViT(image_size=32,
                    num_channels=3, 
                    patch_size=4,
                    num_classes=num_classes,
                    hidden_dim=256,
                    num_transformer_layers=6,
                    num_head=4,
                    feedforward_dim=1024,
                    dropout_prob=0.1,)
    else:
        raise Exception("Model not supported: {}".format(model_name))
    
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Using model {model_name} with {total_params} parameters ({trainable_params} trainable)")

    return model

In [17]:
def train_main(config):
    ## data and preprocessing settings
    data_root_dir = config['data_root_dir']
    num_worker = config.get('num_worker', 4)

    ## Hyper parameters
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    start_epoch = config.get('start_epoch', 0)
    num_epochs = config['num_epochs']

    ## checkpoint setting
    checkpoint_save_interval = config.get('checkpoint_save_interval', 10)
    checkpoint_path = config.get('checkpoint_path', "checkpoints/checkpoint.pth")
    best_model_path = config.get('best_model_path', "checkpoints/best_model.pth")
    load_from_checkpoint = config.get('load_from_checkpoint', None)

    ## variables
    best_metric = 0

    wandb.init(
        project=config["wandb_project_name"],
        config=config
    )

    device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    print(f"Using {device} device")

    train_dataloader, val_dataloader, test_dataloader, num_classes = load_cifar10_dataloaders(
        data_root_dir, device, batch_size = batch_size, num_worker = num_worker)
    
    model = get_model(model_name = config["model_name"], num_classes= num_classes, config = config).to(device)

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1) 

    if load_from_checkpoint:
        load_checkpoint_path = (best_model_path if load_from_checkpoint == "best" else checkpoint_path)
        start_epoch, best_metric = load_checkpoint(load_checkpoint_path, model, optimizer, scheduler, device)

    if config.get('test_mode', False):
        # Only evaluate on the test dataset
        print("Running test evaluation...")
        test_metric = evaluation_loop(model, device, test_dataloader, criterion, phase = "test")
        print(f"Test Accuracy: {test_metric}")
        
    else:
        # Train and validate using train/val datasets
        for epoch in range(start_epoch, num_epochs):
            train_loop(model, device, train_dataloader, criterion, optimizer, epoch)
            val_metric = evaluation_loop(model, device, val_dataloader, criterion, epoch = epoch, phase = "validation")
            scheduler.step()

            if (epoch + 1) % checkpoint_save_interval == 0 or (epoch + 1) == num_epochs:
                is_best = val_metric > best_metric
                best_metric = max(val_metric, best_metric)
                save_checkpoint(checkpoint_path, model, optimizer, scheduler, epoch, best_metric, is_best, best_model_path)



    wandb.finish()


In [18]:
config = {
    'data_root_dir': '/datasets',
    'batch_size': 16,
    'learning_rate': 1e-3,
    'model_name': 'ViT',
    'num_epochs': 2,

    "dataset": "CIFAR-10",
    'wandb_project_name': 'ViT-CIFAR',

    "checkpoint_save_interval" : 10,
    "checkpoint_path" : "checkpoints/checkpoint.pth",
    "best_model_path" : "checkpoints/best_model.pth",
    "load_from_checkpoint" : None,    # Options: "latest", "best", or None
}

In [ ]:
train_main(config)

#### 저장된 checkpoint를 모두 지워 저장공간을 확보한다

In [ ]:
import shutil, os
if os.path.exists('checkpoints/'):
    shutil.rmtree('checkpoints/')